# Spark and Elasticsearch Integration

<b>Este bloco de notas demonstra como você pode usar o Spark para ler dados de streaming, realizar uma transformação nos dados e gravar os novos dados no Elasticsearch.

Também demonstra como você pode usar o Spark para ler um índice do Elasticsearch.
</b>

# Instalando o ES-Hadoop

<b>Primeiro, precisamos instalar o conector Elastic-hadoop. 
Isso deve ser instalado no mesmo caminho em todo o cluster.</b>

In [1]:
import os
import urllib.request
import zipfile

# Obtendo a configuração do Elasticsearch

In [2]:
#print(os.listdir('../../file/'))
#os.chdir('../../file/')

#!pip install elasticsearch


In [3]:
from elasticsearch import Elasticsearch

<b> Primeiro configure a conexão do Elasticsearch por padrão, nos conectamos ao elasticsearch: 9200, como estamos executando este notebook no Spark-Node, precisamos usar 'elasticsearch' em vez de 'localhost', pois esse é o nome do docker ontainer executando o Elasticsearch. Se o índice de teste de fluxo existir, limpe-o e crie um novo. </b>

In [4]:
es = Elasticsearch('elasticsearch-node:9200')
#es = Elasticsearch('http://localhost:9200/')


if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    es.indices.create('stream-test')
    

# Spark Streaming para Elasticsearch

<b> Precisamos garantir que o conector ES-Hadoop esteja no caminho de classe do driver. </b>

In [5]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

print(os.getcwd())

import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'

/home/jovyan


<b> Agora podemos criar nosso Contexto Spark (é o ponto de entrada para qualquer funcionalidade do spark). </b>

In [6]:
sc = SparkContext(appName="PythonSparkStreaming")  
sc.setLogLevel("WARN")

<b>...e nosso contexto de streaming ( representa a conexão com um cluster Spark e pode ser usado para criar várias fontes de entrada). </b>

In [7]:
ssc = StreamingContext(sc, 3)

<b> Em seguida, vamos gerar um fluxo de arquivos. Nesse caso, transmitiremos todos os arquivos gravados no diretório de amostra. Isso está sendo bombeado com dados aleatórios.</b>

In [8]:
stream = ssc.textFileStream('sample/')
print(os.listdir(os.getcwd()+'/sample/'))

['stream-sample0.48996529269460665.txt', 'stream-sample0.4162715203451266.txt', 'stream-sample0.6237267281963705.txt', 'stream-sample0.4230371416159896.txt', 'stream-sample0.8110266430648445.txt', 'stream-sample0.7432590129200752.txt', 'stream-sample0.6538513847601882.txt', 'stream-sample0.7684445710038628.txt', 'stream-sample0.6429574543945119.txt', 'stream-sample0.06883720464154963.txt', 'stream-sample0.6015869204078964.txt', 'stream-sample0.7249063761272927.txt', 'stream-sample0.11183131085256448.txt', 'stream-sample0.5916542311610035.txt', 'stream-sample0.577695830294339.txt', 'stream-sample0.310789032340671.txt']


<b>Vamos querer realizar algumas transformações leves nesses dados. Primeiramente, queremos analisar o horário da época em uma sequência de datas que o Elasticsearch possa entender.</b>

In [9]:
def format_sample(x):
    data = json.loads(x)
    data['timestamp'] = datetime.fromtimestamp(data['timestamp']).strftime('%Y/%m/%d %H:%M:%S')
    data['doc_id'] = data.pop('count')
    return (data['doc_id'], json.dumps(data))

<b>Mapearemos a função que acabamos de criar para o fluxo, para que cada registro, em cada lote, seja analisado com a mesma função.</b>

In [10]:
parsed = stream.map(lambda x: format_sample(x))

<b> Em seguida, vamos definir uma função que grave o RDD gerado por cada operação em lote de streaming no Elasticsearch. </b>

In [11]:
def handler(rdd):
        es_write_conf = {
        # especifique o nó para o qual estamos enviando dados (esse deve ser o mestre)
        "es.nodes" : 'aacde1e0f861',
            
        # especifique a porta caso ela não seja a porta padrão
        "es.port" : '9200',
            
        # especifique um recurso no formato '<index>/<type>'
        "es.resource" : 'stream-test',

        # e a entrada JSON?
        "es.input.json" : "true",
            
        # O campo do documento / nome da propriedade que contém o ID do documento.
        "es.mapping.id": "_id",
        }

        rdd.saveAsNewAPIHadoopFile(
                path='/home/jovyan',
                outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
                keyClass="org.apache.hadoop.io.NullWritable",
                valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",

                # criticamente, devemos especificar nosso 'es_write_conf' 
                conf=es_write_conf)

<b>Agora podemos aplicar nosso manipulador a cada registro transmitido pelo sistema.</b>

In [12]:
parsed.foreachRDD(lambda rdd: handler(rdd))

<b>Podemos solicitar que também seja impresso em 'stdout'.</b>

In [13]:
parsed.pprint()

<b>Finalmente, podemos começar o contexto do spark.</b>

In [14]:
ssc.start()

-------------------------------------------
Time: 2020-02-04 20:43:48
-------------------------------------------

-------------------------------------------
Time: 2020-02-04 20:43:51
-------------------------------------------

-------------------------------------------
Time: 2020-02-04 20:43:54
-------------------------------------------

-------------------------------------------
Time: 2020-02-04 20:43:57
-------------------------------------------

-------------------------------------------
Time: 2020-02-04 20:44:00
-------------------------------------------



<b>Se queremos parar o contexto, precisamos chamar o método 'stop'</b>

In [15]:
ssc.stop()

# Processamento em massa ES com Spark (Bulk)

<b>Vamos gerar um novo contexto do spark desde que matamos nosso último. Usaremos isso para operar em um índice inteiro de dados de ES usando o Spark. Nesse caso, leremos novamente os dados que acabamos de enviar usando o Spark Streaming.</b>

In [16]:
sc = SparkContext(appName="PythonSparkReading")  
sc.setLogLevel("WARN")
#sc.stop()

In [18]:
es_read_conf = { 
    # especifique o nó para o qual estamos enviando dados (esse deve ser o mestre)    
    "es.nodes" : "aacde1e0f861",
    
    # especifique o recurso de leitura no formato '<index>/<type>'
    "es.resource" : "stream-test"
    }

es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf=es_read_conf)

<b>Vamos pegar uma amostra dos dados que acabamos de extrair do Elasticsearch. Observe que cada objeto é uma tupla, onde o primeiro item é o ID do documento.</b>

In [19]:

es_rdd.take(5)

[]

<b>Vamos converter essas tuplas em JSON puro</b>

In [ ]:
es_rdd = es_rdd.map(lambda x: x[1])

In [ ]:
es_rdd.take(1)

<b>Agora, vamos converter o RDD em um Dataframe Spark SQL para que possamos tratá-lo mais como um objeto Pandas</b>

In [ ]:
from pyspark.sql import SparkSession, SQLContext, Row

# Executa um 'monkey patch' no contexto do Spark, estendendo-o com recursos do Spark SQL
spark = SparkSession \
    .builder \
    .appName("Spark SQL") \
    .getOrCreate()

In [ ]:
df = es_rdd.map(lambda l: Row(**dict(l))).toDF()

In [ ]:
df.take(1)

<b> Podemos executar um 'groupby' nos dados. Nesse caso, agruparemos por nome </b>

In [ ]:
df \
    .groupby('name') \
    .count() \
    .collect()

<b>Também podemos filtrar dados como no 'Pandas'.</b>

In [ ]:
df \
    .filter(df.name == 'Samwise')\
    .take(1)

<b>Novamente, precisamos parar o contexto, se quisermos voltar para um Contexto de Streaming</b>

In [ ]:
sc.stop()